# Workflow

In [1]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import sys

sys.path.append('../hydra_tod/')

from astropy.coordinates import EarthLocation, AltAz, SkyCoord
from astropy.time import Time, TimeDelta
import astropy.units as u
from utils import Leg_poly_proj, view_samples
from flicker_model import sim_noise, flicker_cov


## A. Simulation

In [2]:
import pickle
from simulation import MultiTODSimulation
# Load the simulation data from a pickle file
with open('multi_tod_simulation_data.pkl', 'rb') as f:
    multi_tod_sim = pickle.load(f)

In [3]:

# Make all the individual variables available for backward compatibility
t_list = multi_tod_sim.t_list
theta_c_setting = multi_tod_sim.theta_c_setting
phi_c_setting = multi_tod_sim.phi_c_setting
theta_c_rising = multi_tod_sim.theta_c_rising
phi_c_rising = multi_tod_sim.phi_c_rising
bool_map_setting = multi_tod_sim.bool_map_setting
bool_map_rising = multi_tod_sim.bool_map_rising
integrated_beam_setting = multi_tod_sim.integrated_beam_setting
integrated_beam_rising = multi_tod_sim.integrated_beam_rising
full_bool_map = multi_tod_sim.full_bool_map
pixel_indices = multi_tod_sim.pixel_indices
integrated_beam = multi_tod_sim.integrated_beam
Tsky_operator_setting = multi_tod_sim.Tsky_operator_setting
Tsky_operator_rising = multi_tod_sim.Tsky_operator_rising
sky_params = multi_tod_sim.sky_params
ntime = multi_tod_sim.ntime
ndiode_proj = multi_tod_sim.ndiode_proj
nd_rec_operator = multi_tod_sim.nd_rec_operator
nd_rec_params = multi_tod_sim.nd_rec_params
gain_proj = multi_tod_sim.gain_proj
gain_params_setting = multi_tod_sim.gain_params_setting
gain_params_rising = multi_tod_sim.gain_params_rising
gains_setting = multi_tod_sim.gains_setting
gains_rising = multi_tod_sim.gains_rising
fc = multi_tod_sim.fc
logfc = multi_tod_sim.logfc
f0 = multi_tod_sim.f0
noise_setting = multi_tod_sim.noise_setting
noise_rising = multi_tod_sim.noise_rising
Tsys_setting = multi_tod_sim.Tsys_setting
Tsys_rising = multi_tod_sim.Tsys_rising
TOD_setting = multi_tod_sim.TOD_setting
TOD_rising = multi_tod_sim.TOD_rising
pixels_c_setting = multi_tod_sim.pixels_c_setting
bool_map_c_setting = multi_tod_sim.bool_map_c_setting
calibration_1_index = multi_tod_sim.calibration_1_index
calibration_5_indices = multi_tod_sim.calibration_5_indices

# Additional constants from the original code
T_ndiode = multi_tod_sim.T_ndiode
rec_params = multi_tod_sim.rec_params
dtime = multi_tod_sim.dtime
alpha = multi_tod_sim.alpha
logf0_list = [multi_tod_sim.logf0]
sigma_2 = multi_tod_sim.sigma_2

In [4]:
init_Tsys_params = np.hstack([sky_params, nd_rec_params, nd_rec_params])
init_noise_params = [-4.8, 2.2]

gain_prior_cov_inv = np.zeros(4)
gain_prior_cov_inv[0] = 2.51 # DC mode: Prior STD ~10%
gain_prior_mean = gain_params_setting

prior_std_frac = 0.1
Tsky_prior_cov_inv = 1. / (prior_std_frac * sky_params)**2
calibration_indices = calibration_1_index
Tsky_prior_cov_inv[calibration_indices] = 1e20

aux = np.ones_like(nd_rec_params)*0.0
aux[0] = 1
#Trec_prior_cov_inv = [aux] # Effectlively no prior

Trec_prior_cov_inv_list = [aux, aux]

def log_prior_noise(params):
    _, alpha_est = params
    return -1e10 * (alpha_est - alpha)**4 

In [5]:
# Import the FlickerCorrEmulator class first
from flicker_model import FlickerCorrEmulator
from full_Gibbs_sampler import full_Gibbs_sampler_multi_TODS_v2

Tsys_samples, all_gain_samples, all_noise_samples = \
    full_Gibbs_sampler_multi_TODS_v2([TOD_setting, TOD_rising],
                                    [t_list, t_list],
                                    [gain_proj, gain_proj],
                                    [Tsky_operator_setting, Tsky_operator_rising],
                                    [nd_rec_operator, nd_rec_operator],
                                    init_Tsys_params,
                                    init_noise_params, 
                                    [logfc, logfc],
                                    num_Jeffrey=False,
                                    wnoise_var=2.5e-6,
                                    Tsky_prior_cov_inv=Tsky_prior_cov_inv,
                                    Tsky_prior_mean=sky_params,
                                    local_Trec_prior_cov_inv_list=Trec_prior_cov_inv_list,
                                    local_Trec_prior_mean_list=[nd_rec_params, nd_rec_params],
                                    local_gain_prior_cov_inv_list=[gain_prior_cov_inv,gain_prior_cov_inv],
                                    local_gain_prior_mean_list=[gain_params_setting,gain_params_rising],
                                    # local_noise_prior_func_list=[log_prior_noise, log_prior_noise],
                                    n_samples=1000,
                                    tol=1e-12)


Using the emulator for flicker noise correlation function.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 0: [6.31182531 0.41930907 0.26437755 0.05666635]
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 0: [6.31182531 0.41930907 0.26437755 0.05666635]
Rank: 0, local id: 0, noise_sample 0: [-4.84793383  2.39961796]
Rank: 0, local id: 0, noise_sample 0: [-4.84793383  2.39961796]
Converged in 5 iterations.
Rank: 0, local id: 1, gain_sample 0: [6.84497078 0.14168205 0.74334535 0.78194953]
Converged in 5 iterations.
Rank: 0, local id: 1, gain_sample 0: [6.84497078 0.14168205 0.74334535 0.78194953]
Rank: 0, local id: 1, noise_sample 0: [-4.55790398  2.44921485]
Rank: 0, local id: 1, noise_sample 0: [-4.55790398  2.44921485]
Reached max iterations with fractional norm error 0.00013248994343964684.
Reached max iterations with fractional norm error 0.00013248994343964684.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 1: [6.31204373 0.41849973 0.26536109

/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)


Rank: 0, local id: 0, noise_sample 4: [-5.09824682  2.11880086]
Converged in 5 iterations.
Rank: 0, local id: 1, gain_sample 4: [6.84494344 0.14252435 0.74567309 0.78212839]
Converged in 5 iterations.
Rank: 0, local id: 1, gain_sample 4: [6.84494344 0.14252435 0.74567309 0.78212839]
Rank: 0, local id: 1, noise_sample 4: [-4.66179175  2.14693327]
Rank: 0, local id: 1, noise_sample 4: [-4.66179175  2.14693327]
Reached max iterations with fractional norm error 0.0015736683973824248.
Reached max iterations with fractional norm error 0.0015736683973824248.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 5: [6.31152026 0.41772966 0.26302107 0.05967884]
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 5: [6.31152026 0.41772966 0.26302107 0.05967884]
Rank: 0, local id: 0, noise_sample 5: [-4.91883147  2.08768113]
Rank: 0, local id: 0, noise_sample 5: [-4.91883147  2.08768113]
Converged in 5 iterations.
Rank: 0, local id: 1, gain_sample 5: [6.84505995 0.14235903 0.74659

/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)


Rank: 0, local id: 1, noise_sample 14: [-4.52227514  2.50400366]
Reached max iterations with fractional norm error 0.0022533573343159977.
Reached max iterations with fractional norm error 0.0022533573343159977.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 15: [6.31263278 0.4197778  0.26723518 0.059781  ]
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 15: [6.31263278 0.4197778  0.26723518 0.059781  ]
Rank: 0, local id: 0, noise_sample 15: [-5.39296898  3.03350157]
Rank: 0, local id: 0, noise_sample 15: [-5.39296898  3.03350157]
Converged in 5 iterations.
Converged in 5 iterations.
Rank: 0, local id: 1, gain_sample 15: [6.84497274 0.14071329 0.74179447 0.78711073]
Rank: 0, local id: 1, gain_sample 15: [6.84497274 0.14071329 0.74179447 0.78711073]
Rank: 0, local id: 1, noise_sample 15: [-5.22195727  1.87806961]
Rank: 0, local id: 1, noise_sample 15: [-5.22195727  1.87806961]
Reached max iterations with fractional norm error 0.001098114619039456.
Reached max i

...

Rank: 0, local id: 1, noise_sample 995: [-4.68676579  2.38189561]
Reached max iterations with fractional norm error 0.0004032209992344825.
Reached max iterations with fractional norm error 0.0004032209992344825.
Converged in 5 iterations.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 996: [6.29428775e+00 4.53787743e-01 2.29897830e-01 5.73365606e-03]
Rank: 0, local id: 0, gain_sample 996: [6.29428775e+00 4.53787743e-01 2.29897830e-01 5.73365606e-03]


/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)
/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)
/opt/homebrew/Caskroom/miniconda/base/envs/TOD/lib/python3.13/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in scalar subtract
  lnpdiff = f + nlp - state.log_prob[j]
/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)
/opt/homebrew/Caskroom/miniconda/base/envs/TOD/lib/python3.13/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in scalar subtract
  lnpdiff = f + nlp - state.log_prob[j]
/Users/zzhang/Workspace/flicker/notebooks

Rank: 0, local id: 0, noise_sample 996: [-3.40245421  2.9198294 ]
Converged in 5 iterations.
Rank: 0, local id: 1, gain_sample 996: [6.83395466 0.13826973 0.73890341 0.77969631]
Converged in 5 iterations.
Rank: 0, local id: 1, gain_sample 996: [6.83395466 0.13826973 0.73890341 0.77969631]
Rank: 0, local id: 1, noise_sample 996: [-5.48744121  1.6391171 ]
Rank: 0, local id: 1, noise_sample 996: [-5.48744121  1.6391171 ]
Reached max iterations with fractional norm error 0.0002398218524881532.
Reached max iterations with fractional norm error 0.0002398218524881532.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 997: [6.29598917 0.41255367 0.26133127 0.05804429]
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 997: [6.29598917 0.41255367 0.26133127 0.05804429]
Rank: 0, local id: 0, noise_sample 997: [-4.38709486  1.99530565]
Rank: 0, local id: 0, noise_sample 997: [-4.38709486  1.99530565]
Converged in 5 iterations.
Rank: 0, local id: 1, gain_sample 997: [6.8345764

/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)
/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)
/opt/homebrew/Caskroom/miniconda/base/envs/TOD/lib/python3.13/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in scalar subtract
  lnpdiff = f + nlp - state.log_prob[j]
/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)
/opt/homebrew/Caskroom/miniconda/base/envs/TOD/lib/python3.13/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in scalar subtract
  lnpdiff = f + nlp - state.log_prob[j]


Rank: 0, local id: 1, noise_sample 997: [-5.98236112  1.57765807]
Reached max iterations with fractional norm error 0.0003143414371903728.
Reached max iterations with fractional norm error 0.0003143414371903728.
Converged in 5 iterations.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 998: [6.2965441  0.41595979 0.26239669 0.05536827]
Rank: 0, local id: 0, gain_sample 998: [6.2965441  0.41595979 0.26239669 0.05536827]
Rank: 0, local id: 0, noise_sample 998: [-4.1737315   2.52685867]
Rank: 0, local id: 0, noise_sample 998: [-4.1737315   2.52685867]
Converged in 5 iterations.
Converged in 5 iterations.
Rank: 0, local id: 1, gain_sample 998: [6.83450882 0.13787594 0.74168106 0.78019363]
Rank: 0, local id: 1, gain_sample 998: [6.83450882 0.13787594 0.74168106 0.78019363]
Rank: 0, local id: 1, noise_sample 998: [-5.07841152  3.19890145]
Rank: 0, local id: 1, noise_sample 998: [-5.07841152  3.19890145]
Reached max iterations with fractional norm error 0.001323602488426521.
Reac

In [6]:
# First, save the samples as npy files..

np.save("outputs/GS1_db/Tsys_samples.npy", Tsys_samples)
np.save("outputs/GS1_db/gain_samples.npy", np.concatenate(all_gain_samples, axis=0))
np.save("outputs/GS1_db/noise_samples.npy", np.concatenate(all_noise_samples, axis=0))